In [1]:
import pandas as pd, numpy as np, datetime as dt, re
import matplotlib.pyplot as plt, scipy.stats as stats
import sqlalchemy as sql

In C:\Users\Kyle\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Kyle\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Kyle\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Kyle\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Kyle\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle

In [2]:
engine = sql.create_engine(f"sqlite:///../data/interim/NBA.db")
conn = engine.connect()
df_all = pd.read_sql("SELECT * FROM basic_stats", con=conn)

In [3]:
df_all.shape

(830021, 26)

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830021 entries, 0 to 830020
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              830021 non-null  int64  
 1   name            830021 non-null  object 
 2   minutes_played  640649 non-null  float64
 3   fg              640649 non-null  float64
 4   fga             640649 non-null  float64
 5   fg_pct          640649 non-null  float64
 6   fg3             640649 non-null  float64
 7   fg3a            640649 non-null  float64
 8   fg3_pct         640649 non-null  float64
 9   ft              640649 non-null  float64
 10  fta             640649 non-null  float64
 11  ft_pct          640649 non-null  float64
 12  orb             640649 non-null  float64
 13  drb             640649 non-null  float64
 14  trb             640649 non-null  float64
 15  ast             640649 non-null  float64
 16  stl             640649 non-null  float64
 17  blk       

In [5]:
# player data is complete for the whole seasons, nulls reference players who were on roster but did not register a second of playing time (dnp column = 1) 

In [7]:
# Timebased Breakdown of All the Stats
df_all.groupby("timetype").count()

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,ast,stl,blk,tov,pf,pts,bpm,dnp,team,game_code
timetype,,,,,,,,,,,,,,,,,,,,,
1H,117497,117497,92342,92342,92342,92342,92342,92342,92342,92342,...,92342,92342,92342,92342,92342,92342,92342,117497,117497,117497
1Q,117497,117497,87255,87255,87255,87255,87255,87255,87255,87255,...,87255,87255,87255,87255,87255,87255,87255,117497,117497,117497
2H,117497,117497,96168,96168,96168,96168,96168,96168,96168,96168,...,96168,96168,96168,96168,96168,96168,96168,117497,117497,117497
2OT,821,821,428,428,428,428,428,428,428,428,...,428,428,428,428,428,428,428,821,821,821
2Q,117497,117497,89137,89137,89137,89137,89137,89137,89137,89137,...,89137,89137,89137,89137,89137,89137,89137,117497,117497,117497
3OT,133,133,73,73,73,73,73,73,73,73,...,73,73,73,73,73,73,73,133,133,133
3Q,117497,117497,85652,85652,85652,85652,85652,85652,85652,85652,...,85652,85652,85652,85652,85652,85652,85652,117497,117497,117497
4OT,54,54,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,54,54,54
4Q,117497,117497,86246,86246,86246,86246,86246,86246,86246,86246,...,86246,86246,86246,86246,86246,86246,86246,117497,117497,117497


In [8]:
df_all.groupby("name").count().sort_values("fg", ascending=False)

,id,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
name,,,,,,,,,,,,,,,,,,,,,
Team Totals,60836,60836,60836,60836,60836,60836,60836,60836,60836,60836,...,60836,60836,60836,60836,60836,60836,60836,60836,60836,60836
Joe Ingles,2181,2151,2151,2151,2151,2151,2151,2151,2151,2151,...,2151,2151,2151,2151,2151,2151,2181,2181,2181,2181
P.J. Tucker,2193,2138,2138,2138,2138,2138,2138,2138,2138,2138,...,2138,2138,2138,2138,2138,2138,2193,2193,2193,2193
Jae Crowder,2254,2119,2119,2119,2119,2119,2119,2119,2119,2119,...,2119,2119,2119,2119,2119,2119,2254,2254,2254,2254
Donovan Mitchell,2138,2092,2092,2092,2092,2092,2092,2092,2092,2092,...,2092,2092,2092,2092,2092,2092,2138,2138,2138,2138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sheldon Mac,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,15,15,15,15
Jaren Jackson Jr.,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,17,17,17,17
Cody Demps,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,14,14,14,14


In [17]:
df_all[(df_all.name != "Team Totals") & ((df_all.timetype == "1H") | (df_all.timetype == "2H"))].groupby(["team", "game_code"]).sum().head()

id  minutes_played    fg   fga  fg_pct   fg3  fg3a  \
team game_code                                                                
ATL  201710180DAL  20546344        240.0001  48.0  94.0  11.805   9.0  18.0   
     201710200CHO  18862090        240.0000  36.0  95.0   6.882   7.0  30.0   
     201710220BRK  15718638        240.0000  32.0  94.0   6.634   7.0  27.0   
     201710230MIA  18351320        240.0001  36.0  76.0   8.494   7.0  17.0   
     201710260CHI  17623892        239.9999  32.0  81.0   6.404  11.0  28.0   

                   fg3_pct    ft   fta  ...   drb   trb   ast   stl  blk  \
team game_code                          ...                                
ATL  201710180DAL    6.000  12.0  15.0  ...  36.0  50.0  20.0  11.0  6.0   
     201710200CHO    4.166  12.0  14.0  ...  32.0  38.0  19.0  11.0  4.0   
     201710220BRK    3.583  33.0  36.0  ...  32.0  48.0  21.0   6.0  2.0   
     201710230MIA    4.833  14.0  18.0  ...  35.0  41.0  14.0   9.0  1.0   
     201710260CHI    4.167  11.0  13.0  ...  33.0  40.0  22.0  10.0  5.0   

                    tov    pf    pts   bpm  dnp  
team game_code                                   
ATL  201710180DAL  13.0  18.0  117.0  30.0    3  
     201710200CHO  13.0  29.0   91.0 -90.0    2  
     201710220BRK  16.0  25.0  104.0 -65.0    2  
     201710230MIA  20.0  16.0   93.0 -55.0    7  
     201710260CHI  10.0  23.0   86.0 -25.0    7  

[5 rows x 22 columns]

In [16]:
df_all[(df_all.name != "Total Totals") & ((df_all.timetype == "1H") | (df_all.timetype == "2H"))].head(35)

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
42,43,Luka Dončić,16.9333,3.0,7.0,0.429,2.0,3.0,0.667,4.0,...,0.0,0.0,2.0,1.0,12.0,-6.0,0,1H,DAL,201810260TOR
43,44,DeAndre Jordan,14.4167,2.0,2.0,1.000,0.0,0.0,0.000,4.0,...,0.0,0.0,0.0,1.0,8.0,-9.0,0,1H,DAL,201810260TOR
44,45,Wesley Matthews,15.9167,6.0,9.0,0.667,2.0,4.0,0.500,0.0,...,0.0,0.0,1.0,1.0,14.0,-2.0,0,1H,DAL,201810260TOR
45,46,Jalen Brunson,16.0667,0.0,4.0,0.000,0.0,1.0,0.000,0.0,...,1.0,0.0,0.0,2.0,0.0,-9.0,0,1H,DAL,201810260TOR
46,47,Harrison Barnes,14.3833,4.0,11.0,0.364,1.0,5.0,0.200,1.0,...,0.0,1.0,0.0,1.0,10.0,-17.0,0,1H,DAL,201810260TOR
47,48,Maxi Kleber,13.1667,2.0,3.0,0.667,1.0,2.0,0.500,1.0,...,1.0,2.0,0.0,0.0,6.0,1.0,0,1H,DAL,201810260TOR
48,49,Dorian Finney-Smith,10.4000,0.0,1.0,0.000,0.0,1.0,0.000,0.0,...,0.0,0.0,0.0,2.0,0.0,-3.0,0,1H,DAL,201810260TOR
49,50,J.J. Barea,9.1333,3.0,7.0,0.429,0.0,0.0,0.000,0.0,...,0.0,0.0,1.0,0.0,6.0,0.0,0,1H,DAL,201810260TOR
50,51,Dwight Powell,9.5833,2.0,2.0,1.000,0.0,0.0,0.000,0.0,...,1.0,0.0,1.0,2.0,4.0,0.0,0,1H,DAL,201810260TOR
51,52,Daryl Macon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,1H,DAL,201810260TOR


In [18]:
# Create a dataframe of all Teams Total Game Stats for each game
df_games = df_all[lambda x: (x.name == "Team Totals") & (x.timetype == "TotalBasics")]

In [19]:
df_games.shape

(8612, 26)

In [20]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8612 entries, 13 to 829936
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              8612 non-null   int64  
 1   name            8612 non-null   object 
 2   minutes_played  8612 non-null   float64
 3   fg              8612 non-null   float64
 4   fga             8612 non-null   float64
 5   fg_pct          8612 non-null   float64
 6   fg3             8612 non-null   float64
 7   fg3a            8612 non-null   float64
 8   fg3_pct         8612 non-null   float64
 9   ft              8612 non-null   float64
 10  fta             8612 non-null   float64
 11  ft_pct          8612 non-null   float64
 12  orb             8612 non-null   float64
 13  drb             8612 non-null   float64
 14  trb             8612 non-null   float64
 15  ast             8612 non-null   float64
 16  stl             8612 non-null   float64
 17  blk             8612 non-null 

In [21]:
# There should be 2 Game Codes per Game (Indicative of 2 teams per Game); Find and Filter Duplicate Games
df_games.groupby("game_code").count().sort_values("fg", ascending=False)[lambda x: x.id > 2]

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,ast,stl,blk,tov,pf,pts,bpm,dnp,timetype,team
game_code,,,,,,,,,,,,,,,,,,,,,
201901210UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902010UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902020UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902230UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201901140UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201902060UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201901230UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201903020UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
201901180UTA,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [22]:
dup_games = set(df_games.groupby("game_code").count().sort_values("fg", ascending=False)[lambda x: x.id > 2].index)

In [24]:
# Take a look at duplicate games; Make sure both teams are equally duplicated for each game
df_games[df_games.game_code.isin(dup_games)].sort_values(["game_code"]).head(20)

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
56289,56290,Team Totals,240.0,37.0,93.0,0.398,16.0,42.0,0.381,18.0,...,7.0,6.0,16.0,16.0,108.0,0.0,0,TotalBasics,UTA,201812190UTA
59894,59895,Team Totals,240.0,37.0,93.0,0.398,16.0,42.0,0.381,18.0,...,7.0,6.0,16.0,16.0,108.0,0.0,0,TotalBasics,UTA,201812190UTA
56197,56198,Team Totals,240.0,38.0,95.0,0.400,10.0,31.0,0.323,17.0,...,10.0,9.0,12.0,21.0,103.0,0.0,0,TotalBasics,GSW,201812190UTA
59802,59803,Team Totals,240.0,38.0,95.0,0.400,10.0,31.0,0.323,17.0,...,10.0,9.0,12.0,21.0,103.0,0.0,0,TotalBasics,GSW,201812190UTA
58277,58278,Team Totals,240.0,42.0,89.0,0.472,9.0,25.0,0.360,14.0,...,11.0,5.0,17.0,21.0,107.0,0.0,0,TotalBasics,OKC,201812220UTA
58669,58670,Team Totals,240.0,42.0,89.0,0.472,9.0,25.0,0.360,14.0,...,11.0,5.0,17.0,21.0,107.0,0.0,0,TotalBasics,OKC,201812220UTA
58767,58768,Team Totals,240.0,42.0,93.0,0.452,11.0,34.0,0.324,11.0,...,12.0,4.0,13.0,19.0,106.0,0.0,0,TotalBasics,UTA,201812220UTA
58375,58376,Team Totals,240.0,42.0,93.0,0.452,11.0,34.0,0.324,11.0,...,12.0,4.0,13.0,19.0,106.0,0.0,0,TotalBasics,UTA,201812220UTA
59992,59993,Team Totals,240.0,35.0,89.0,0.393,9.0,23.0,0.391,17.0,...,7.0,3.0,12.0,20.0,96.0,0.0,0,TotalBasics,POR,201812250UTA
60090,60091,Team Totals,240.0,46.0,83.0,0.554,13.0,29.0,0.448,12.0,...,7.0,10.0,17.0,19.0,117.0,0.0,0,TotalBasics,UTA,201812250UTA


In [25]:
# Drop duplicates
df_games.drop_duplicates(["team", "game_code"], inplace=True)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df_games.groupby("game_code").count()

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,ast,stl,blk,tov,pf,pts,bpm,dnp,timetype,team
game_code,,,,,,,,,,,,,,,,,,,,,
201710170CLE,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710170GSW,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710180BOS,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710180DAL,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
201710180DET,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202103040NYK,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
202103040PHO,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
202103040POR,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [27]:
df_games.head()

,id,name,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,stl,blk,tov,pf,pts,bpm,dnp,timetype,team,game_code
13,14,Team Totals,240.0,38.0,92.0,0.413,12.0,30.0,0.400,19.0,...,6.0,6.0,14.0,17.0,107.0,0.0,0,TotalBasics,DAL,201810260TOR
109,110,Team Totals,240.0,44.0,91.0,0.484,11.0,27.0,0.407,17.0,...,10.0,5.0,12.0,22.0,116.0,0.0,0,TotalBasics,TOR,201810260TOR
195,196,Team Totals,240.0,40.0,99.0,0.404,14.0,36.0,0.389,7.0,...,4.0,6.0,14.0,27.0,101.0,0.0,0,TotalBasics,BOS,201810190TOR
293,294,Team Totals,240.0,43.0,92.0,0.467,13.0,35.0,0.371,14.0,...,3.0,3.0,12.0,19.0,113.0,0.0,0,TotalBasics,TOR,201810190TOR
391,392,Team Totals,240.0,34.0,85.0,0.400,7.0,19.0,0.368,29.0,...,4.0,0.0,16.0,25.0,104.0,0.0,0,TotalBasics,CLE,201810170TOR


In [29]:
# Create Regular Expressions to Parse the game_code column unique to BBall reference
game_coder = re.compile("\d{8}") # game date
home_team_regex = re.compile("[A-Z]{3}") # home team

In [30]:
sample_string = "202011030"

In [32]:
sample_string[4:6]

'11'

In [33]:
def parse_date(row):
    date = game_coder.search(row)[0]
    return dt.datetime(int(date[:4]), int(date[4:6]), int(date[6:]))

In [34]:
df_games = df_games.assign(date = lambda x: x.game_code.apply(parse_date))

In [35]:
def home_team(row):
    home = home_team_regex.search(row)[0]
    return home

In [36]:
df_games = df_games.assign(home_team = lambda x: x.game_code.apply(home_team))

In [37]:
df_games = df_games.assign(home_team_bin = lambda x: x.home_team == x.team)

In [43]:
# Gather Advanced Stats from other table
df_adv = pd.read_sql("SELECT * FROM advanced_stats", con=conn)

In [44]:
df_adv = df_adv[df_adv.name == "Team Totals"]

In [45]:
df_adv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8612 entries, 13 to 117496
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                8612 non-null   int64  
 1   name              8612 non-null   object 
 2   minutes_played    8612 non-null   float64
 3   ts_pct            8612 non-null   float64
 4   efg_pct           8612 non-null   float64
 5   fg3a_per_fga_pct  8612 non-null   float64
 6   fta_per_fga_pct   8612 non-null   float64
 7   orb_pct           8612 non-null   float64
 8   drb_pct           8612 non-null   float64
 9   trb_pct           8612 non-null   float64
 10  ast_pct           8612 non-null   float64
 11  stl_pct           8612 non-null   float64
 12  blk_pct           8612 non-null   float64
 13  tov_pct           8612 non-null   float64
 14  usg_pct           8612 non-null   float64
 15  off_rtg           8612 non-null   float64
 16  def_rtg           8612 non-null   float

In [46]:
df_adv.shape

(8612, 22)

In [47]:
dup_games_adv = set(df_adv.groupby("game_code").count().sort_values("minutes_played", ascending=False)[lambda x: x.id > 2].index)

In [51]:
# Perform same check as above on Set of Duplicated Game Codes
df_adv[df_adv.game_code.isin(dup_games_adv)].sort_values(["game_code"]).head(20)

,id,name,minutes_played,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,...,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,bpm,dnp,timetype,team,game_code
7990,7991,Team Totals,240.0,0.526,0.484,0.452,0.237,20.8,80.8,52.0,...,9.4,13.5,100.0,104.0,99.1,0.0,0,Advanced,UTA,201812190UTA
8505,8506,Team Totals,240.0,0.526,0.484,0.452,0.237,20.8,80.8,52.0,...,9.4,13.5,100.0,104.0,99.1,0.0,0,Advanced,UTA,201812190UTA
7976,7977,Team Totals,240.0,0.500,0.453,0.326,0.189,19.2,79.2,48.0,...,17.6,10.4,100.0,99.1,104.0,0.0,0,Advanced,GSW,201812190UTA
8491,8492,Team Totals,240.0,0.500,0.453,0.326,0.189,19.2,79.2,48.0,...,17.6,10.4,100.0,99.1,104.0,0.0,0,Advanced,GSW,201812190UTA
8274,8275,Team Totals,240.0,0.542,0.522,0.281,0.247,19.0,81.1,53.7,...,8.5,14.7,100.0,102.8,101.9,0.0,0,Advanced,OKC,201812220UTA
8330,8331,Team Totals,240.0,0.542,0.522,0.281,0.247,19.0,81.1,53.7,...,8.5,14.7,100.0,102.8,101.9,0.0,0,Advanced,OKC,201812220UTA
8344,8345,Team Totals,240.0,0.525,0.511,0.366,0.194,18.9,81.0,46.3,...,6.3,11.4,100.0,101.9,102.8,0.0,0,Advanced,UTA,201812220UTA
8288,8289,Team Totals,240.0,0.525,0.511,0.366,0.194,18.9,81.0,46.3,...,6.3,11.4,100.0,101.9,102.8,0.0,0,Advanced,UTA,201812220UTA
8519,8520,Team Totals,240.0,0.478,0.444,0.258,0.292,21.6,77.5,46.2,...,5.6,10.7,100.0,96.4,117.5,0.0,0,Advanced,POR,201812250UTA
8533,8534,Team Totals,240.0,0.628,0.633,0.349,0.277,22.5,78.4,53.8,...,15.2,15.4,100.0,117.5,96.4,0.0,0,Advanced,UTA,201812250UTA


In [52]:
# They are the same. Drop duplicates
df_adv = df_adv.drop_duplicates(["team", "game_code"])

In [53]:
df_adv.shape

(8572, 22)

In [54]:
df_adv.columns

Index(['id', 'name', 'minutes_played', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'dnp', 'timetype', 'team', 'game_code'],
      dtype='object')

In [55]:
df_games.columns

Index(['id', 'name', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'bpm', 'dnp', 'timetype', 'team',
       'game_code', 'date', 'home_team', 'home_team_bin'],
      dtype='object')

In [56]:
df_games = df_games.loc[:, ['team','minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'home_team', 'home_team_bin', 'date', 'game_code']]

In [57]:
df_adv = df_adv.loc[:, ['team', "game_code", 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm']]

In [58]:
df_games = df_games.reset_index(drop=True)
df_adv = df_adv.reset_index(drop=True)

In [59]:
# Full dataframe combining basic and advanced statistics for teams
df_full_stats = pd.merge(df_games, df_adv, on=["game_code", "team"])

In [60]:
df_full_stats.columns

Index(['team', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'home_team', 'home_team_bin', 'date',
       'game_code', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct',
       'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct',
       'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm'],
      dtype='object')

In [65]:
# Rearrange Columns
df_full_stats = df_full_stats.loc[:, ['team', 'minutes_played', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a',
       'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl',
       'blk', 'tov', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct',
       'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct',
       'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'pf','pts', 'home_team', 'home_team_bin','game_code', 'date']]

In [66]:
# Ensure Symmetry (home teams = away teams)
df_full_stats.groupby("home_team_bin").count()

,team,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,pf,pts,home_team,game_code,date
home_team_bin,,,,,,,,,,,,,,,,,,,,,
False,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286,...,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286
True,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286,...,4286,4286,4286,4286,4286,4286,4286,4286,4286,4286


In [67]:
df_full_stats.head()

,team,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,tov_pct,usg_pct,off_rtg,def_rtg,pf,pts,home_team,home_team_bin,game_code,date
0,DAL,240.0,38.0,92.0,0.413,12.0,30.0,0.400,19.0,24.0,...,12.0,100.0,106.2,115.2,17.0,107.0,TOR,False,201810260TOR,2018-10-26
1,TOR,240.0,44.0,91.0,0.484,11.0,27.0,0.407,17.0,20.0,...,10.7,100.0,115.2,106.2,22.0,116.0,TOR,True,201810260TOR,2018-10-26
2,BOS,240.0,40.0,99.0,0.404,14.0,36.0,0.389,7.0,10.0,...,11.9,100.0,103.3,115.6,27.0,101.0,TOR,False,201810190TOR,2018-10-19
3,TOR,240.0,43.0,92.0,0.467,13.0,35.0,0.371,14.0,19.0,...,10.7,100.0,115.6,103.3,19.0,113.0,TOR,True,201810190TOR,2018-10-19
4,CLE,240.0,34.0,85.0,0.400,7.0,19.0,0.368,29.0,39.0,...,13.5,100.0,103.7,115.6,25.0,104.0,TOR,False,201810170TOR,2018-10-17


In [68]:
# NBA 17-18 Season: Oct 17, 2017 - June 8, 2018
# NBA 18-19 Season: October 16, 2018 - June 13, 2019
# NBA 19-20 Season: October 22, 2019 - October 11, 2020 (Bubble Season) - 71-75 game regular season 
# NBA 20-21 Season: Dec 22, 2020 - July 20th 2021 (Shortened Season) - 72 game regular season + 6 play-in playoff games
def seasoner(date):
    if date <= dt.datetime(2018, 6, 8):
        return "17-18"
    elif date <= dt.datetime(2019, 6, 13):
        return "18-19"
    elif date <= dt.datetime(2020, 10, 11):
        return "19-20"
    else:
        return "20-21"

In [69]:
df_full_stats.assign(NBA_season = lambda x: x.date.apply(seasoner)).groupby("NBA_season").count()

,team,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,tov_pct,usg_pct,off_rtg,def_rtg,pf,pts,home_team,home_team_bin,game_code,date
NBA_season,,,,,,,,,,,,,,,,,,,,,
17-18,2624,2624,2624,2624,2624,2624,2624,2624,2624,2624,...,2624,2624,2624,2624,2624,2624,2624,2624,2624,2624
18-19,2624,2624,2624,2624,2624,2624,2624,2624,2624,2624,...,2624,2624,2624,2624,2624,2624,2624,2624,2624,2624
19-20,2284,2284,2284,2284,2284,2284,2284,2284,2284,2284,...,2284,2284,2284,2284,2284,2284,2284,2284,2284,2284
20-21,1040,1040,1040,1040,1040,1040,1040,1040,1040,1040,...,1040,1040,1040,1040,1040,1040,1040,1040,1040,1040


In [71]:
df_full_stats = df_full_stats.assign(NBA_season = lambda x: x.date.apply(seasoner))

In [72]:
# def playoffs(date):
#     # 4/11/18
#     if (date >= dt.datetime(2018, 4, 11)) and (date <= dt.datetime(2018, 6, 8)):
#         return 1
#     # 4/13/19
#     elif (date >= dt.datetime(2019, 4, 13)) and (date <= dt.datetime(2019, 6, 13)):
#         return 1
#     # 8/17/20
#     elif (date >= dt.datetime(2020, 8, 15)) and (date <= dt.datetime(2020, 10, 11)):
#         return 1
#     else:
#         return 0

In [135]:
# df_full_stats = df_full_stats.assign(playoffs = lambda x: x.date.apply(playoffs))

In [73]:
df_full_stats.head()

,team,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,usg_pct,off_rtg,def_rtg,pf,pts,home_team,home_team_bin,game_code,date,NBA_season
0,DAL,240.0,38.0,92.0,0.413,12.0,30.0,0.400,19.0,24.0,...,100.0,106.2,115.2,17.0,107.0,TOR,False,201810260TOR,2018-10-26,18-19
1,TOR,240.0,44.0,91.0,0.484,11.0,27.0,0.407,17.0,20.0,...,100.0,115.2,106.2,22.0,116.0,TOR,True,201810260TOR,2018-10-26,18-19
2,BOS,240.0,40.0,99.0,0.404,14.0,36.0,0.389,7.0,10.0,...,100.0,103.3,115.6,27.0,101.0,TOR,False,201810190TOR,2018-10-19,18-19
3,TOR,240.0,43.0,92.0,0.467,13.0,35.0,0.371,14.0,19.0,...,100.0,115.6,103.3,19.0,113.0,TOR,True,201810190TOR,2018-10-19,18-19
4,CLE,240.0,34.0,85.0,0.400,7.0,19.0,0.368,29.0,39.0,...,100.0,103.7,115.6,25.0,104.0,TOR,False,201810170TOR,2018-10-17,18-19


## Basketball Reference Data Status:
    *We have a complete dataframe that includes all of the total results based statistics, both advanced and basic, in a single dataframe (df_full_stats).
    
    *The data is complete for all teams and needs to be combined with the game-level bookie information we have from external sources.
    
    *The corresponding join for df_full_stats will be a many-to-one join (2 teams to 1 market row).
    
    * Teams are abbrevaited by their 3 letter bball reference code.

<h1 style="color: green">Offshore Sportsbook Data</h1>

In [74]:
# # Finished pairing Down the Data to a Game by Game table structured by each home team
# g_by_g_df.to_csv("../data/interim/game_by_game_stats.csv", index=False)

In [78]:
# Goal: Pair clean and complete Bookmaker Data With Right Games:
odds_17 = pd.read_csv("../data/external/vegas_17_18.csv")
odds_17_playoffs = pd.read_csv("../data/external/vegas_playoff_17_18.csv")
odds_18 = pd.read_csv("../data/external/vegas_18_19.csv")
odds_18_playoffs = pd.read_csv("../data/external/vegas_playoff_18_19.csv")

In [76]:
odds_17.columns

Index(['Date', 'Location', 'Team', 'OppTeam', 'TeamId', 'GameId',
       'PercentBet_ML', 'Open_Line_ML', 'Pinnacle_ML', '5dimes_ML',
       'Heritage_ML', 'Bovada_ML', 'Betonline_ML', 'Average_Line_ML',
       'Best_Line_ML', 'Worst_Line_ML', 'PercentBet_Spread',
       'Open_Line_Spread', 'Open_Odds_Spread', 'Pinnacle_Line_Spread',
       'Pinnacle_Odds_Spread', '5dimes_Line_Spread', '5dimes_Odds_Spread',
       'Heritage_Line_Spread', 'Heritage_Odds_Spread', 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Betonline_Line_Spread', 'Betonline_Odds_Spread',
       'Average_Line_Spread', 'Average_Odds_Spread', 'Best_Line_Spread',
       'Worst_Line_Spread', 'Best_Odds_Spread', 'Worst_Odds_Spread',
       'PercentBet_OU', 'Open_Line_OU', 'Open_Odds_OU', 'Pinnacle_Line_OU',
       'Pinnacle_Odds_OU', '5dimes_Line_OU', '5dimes_Odds_OU',
       'Heritage_Line_OU', 'Heritage_Odds_OU', 'Bovada_Line_OU',
       'Bovada_Odds_OU', 'Betonline_Line_OU', 'Betonline_Odds_OU',
       'Average_Line

In [77]:
odds_17.head()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Average_Line_OU,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total
0,2017-10-17,away,Boston,Cleveland,1610612738,21700001,50.4,150.0,166,170,...,216.17,-106.83,216.5,216.0,-101.0,-110,99.0,-3.0,L,201.0
1,2017-10-17,home,Cleveland,Boston,1610612739,21700001,49.6,-170.0,-185,-185,...,216.17,-108.17,216.5,216.0,-105.0,-110,102.0,3.0,W,201.0
2,2017-10-17,away,Houston,Golden State,1610612745,21700002,52.7,405.0,377,420,...,231.75,-109.67,235.0,230.5,-103.0,-115,122.0,1.0,W,243.0
3,2017-10-17,home,Golden State,Houston,1610612744,21700002,47.3,-500.0,-440,-475,...,231.75,-105.33,235.0,230.5,-100.0,-110,121.0,-1.0,L,243.0
4,2017-10-18,away,Charlotte,Detroit,1610612766,21700003,51.3,130.0,127,130,...,202.17,-107.83,203.0,202.0,-102.0,-115,90.0,-12.0,L,192.0


In [79]:
odds_17.columns == odds_17_playoffs.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [80]:
odds_17.loc[:, ["playoffs"]] = 0
odds_18.loc[:, ["playoffs"]] = 0
odds_17_playoffs.loc[:, ["playoffs"]] = 1
odds_18_playoffs.loc[:, ["playoffs"]] = 1

In [124]:
odds_18.shape

(2452, 59)

In [81]:
full_odds_df = pd.concat([odds_17, odds_17_playoffs, odds_18, odds_18_playoffs], axis=0, ignore_index=True)

In [85]:
full_odds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5240 entries, 0 to 5239
Data columns (total 59 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   5240 non-null   object 
 1   Location               5240 non-null   object 
 2   Team                   5240 non-null   object 
 3   OppTeam                5240 non-null   object 
 4   TeamId                 5240 non-null   int64  
 5   GameId                 5240 non-null   int64  
 6   PercentBet_ML          5240 non-null   float64
 7   Open_Line_ML           5098 non-null   float64
 8   Pinnacle_ML            5140 non-null   float64
 9   5dimes_ML              5142 non-null   float64
 10  Heritage_ML            5128 non-null   float64
 11  Bovada_ML              5126 non-null   float64
 12  Betonline_ML           5138 non-null   float64
 13  Average_Line_ML        5240 non-null   float64
 14  Best_Line_ML           5240 non-null   float64
 15  Wors

<h3 style="color: red">Understanding Nulls</h3>
<ul>
    <li>The Sportsbook Columns for the variety of lines they provide (moneylines, spreads, over/unders) are the columns with the most nulls</li>
    <li>Additionally, the time based metric of the summary ones (Open_Line_Spread, Open_Line_ML, etc.), which corresponds to what the bookmakers started the market at, is missing data.</li>
    <li>Summary Data about min/max (Best/Worst Line and Odds), Averages, Percentage Bet, nominal Game Data, and results Data are all intact.</li>
</ul>

In [90]:
full_odds_df.iloc[:, 7:13].count()

Open_Line_ML    5098
Pinnacle_ML     5140
5dimes_ML       5142
Heritage_ML     5128
Bovada_ML       5126
Betonline_ML    5138
dtype: int64

In [101]:
full_odds_df[(full_odds_df.iloc[:, 7].isnull())].iloc[:, 7:13].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 1922 to 5201
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open_Line_ML  0 non-null      float64
 1   Pinnacle_ML   134 non-null    float64
 2   5dimes_ML     132 non-null    float64
 3   Heritage_ML   136 non-null    float64
 4   Bovada_ML     140 non-null    float64
 5   Betonline_ML  126 non-null    float64
dtypes: float64(6)
memory usage: 7.8 KB


In [104]:
full_odds_df[(full_odds_df.iloc[:, 8].isnull())].iloc[:, 7:13].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 2726 to 5231
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open_Line_ML  92 non-null     float64
 1   Pinnacle_ML   0 non-null      float64
 2   5dimes_ML     86 non-null     float64
 3   Heritage_ML   84 non-null     float64
 4   Bovada_ML     96 non-null     float64
 5   Betonline_ML  82 non-null     float64
dtypes: float64(6)
memory usage: 5.5 KB


In [105]:
full_odds_df[(full_odds_df.iloc[:, 9].isnull())].iloc[:, 7:13].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 2720 to 5235
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open_Line_ML  88 non-null     float64
 1   Pinnacle_ML   84 non-null     float64
 2   5dimes_ML     0 non-null      float64
 3   Heritage_ML   90 non-null     float64
 4   Bovada_ML     92 non-null     float64
 5   Betonline_ML  74 non-null     float64
dtypes: float64(6)
memory usage: 5.4 KB


In [106]:
full_odds_df[(full_odds_df.iloc[:, 36].isnull())].iloc[:, 36:47].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 2634 to 5235
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Open_Line_OU       0 non-null      float64
 1   Open_Odds_OU       0 non-null      float64
 2   Pinnacle_Line_OU   182 non-null    float64
 3   Pinnacle_Odds_OU   182 non-null    float64
 4   5dimes_Line_OU     183 non-null    float64
 5   5dimes_Odds_OU     183 non-null    float64
 6   Heritage_Line_OU   186 non-null    float64
 7   Heritage_Odds_OU   186 non-null    float64
 8   Bovada_Line_OU     186 non-null    float64
 9   Bovada_Odds_OU     186 non-null    float64
 10  Betonline_Line_OU  184 non-null    float64
dtypes: float64(11)
memory usage: 18.0 KB


In [109]:
full_odds_df[(full_odds_df.iloc[:, 44].isnull())].iloc[:, 36:48].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 2746 to 5235
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Open_Line_OU       136 non-null    float64
 1   Open_Odds_OU       136 non-null    float64
 2   Pinnacle_Line_OU   128 non-null    float64
 3   Pinnacle_Odds_OU   128 non-null    float64
 4   5dimes_Line_OU     116 non-null    float64
 5   5dimes_Odds_OU     116 non-null    float64
 6   Heritage_Line_OU   134 non-null    float64
 7   Heritage_Odds_OU   134 non-null    float64
 8   Bovada_Line_OU     0 non-null      float64
 9   Bovada_Odds_OU     0 non-null      float64
 10  Betonline_Line_OU  122 non-null    float64
 11  Betonline_Odds_OU  122 non-null    float64
dtypes: float64(12)
memory usage: 14.4 KB


In [115]:
full_odds_df[full_odds_df.Bovada_Line_OU.notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5098 entries, 0 to 5239
Data columns (total 59 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   5098 non-null   object 
 1   Location               5098 non-null   object 
 2   Team                   5098 non-null   object 
 3   OppTeam                5098 non-null   object 
 4   TeamId                 5098 non-null   int64  
 5   GameId                 5098 non-null   int64  
 6   PercentBet_ML          5098 non-null   float64
 7   Open_Line_ML           4960 non-null   float64
 8   Pinnacle_ML            5012 non-null   float64
 9   5dimes_ML              5008 non-null   float64
 10  Heritage_ML            4996 non-null   float64
 11  Bovada_ML              4996 non-null   float64
 12  Betonline_ML           5000 non-null   float64
 13  Average_Line_ML        5098 non-null   float64
 14  Best_Line_ML           5098 non-null   float64
 15  Wors

In [200]:
full_odds_df[(full_odds_df.iloc[:, 11].isnull())].loc[:, ["Bovada_ML", 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Bovada_Line_OU',
       'Bovada_Odds_OU']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 586 to 5239
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Bovada_ML           0 non-null      float64
 1   Bovada_Line_Spread  110 non-null    float64
 2   Bovada_Odds_Spread  110 non-null    float64
 3   Bovada_Line_OU      102 non-null    float64
 4   Bovada_Odds_OU      102 non-null    float64
dtypes: float64(5)
memory usage: 5.3 KB


In [201]:
full_odds_df[(full_odds_df.iloc[:, 44].isnull())].loc[:, ["Bovada_ML", 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Bovada_Line_OU',
       'Bovada_Odds_OU']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 2746 to 5235
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Bovada_ML           130 non-null    float64
 1   Bovada_Line_Spread  134 non-null    float64
 2   Bovada_Odds_Spread  134 non-null    float64
 3   Bovada_Line_OU      0 non-null      float64
 4   Bovada_Odds_OU      0 non-null      float64
dtypes: float64(5)
memory usage: 6.7 KB


In [204]:
full_odds_df[(full_odds_df.iloc[:, 25].isnull())].loc[:, ["Bovada_ML", 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Bovada_Line_OU',
       'Bovada_Odds_OU', 'Pinnacle_Line_Spread', 'Pinnacle_Odds_Spread']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 2726 to 5193
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Bovada_ML             50 non-null     float64
 1   Bovada_Line_Spread    0 non-null      float64
 2   Bovada_Odds_Spread    0 non-null      float64
 3   Bovada_Line_OU        46 non-null     float64
 4   Bovada_Odds_OU        46 non-null     float64
 5   Pinnacle_Line_Spread  48 non-null     float64
 6   Pinnacle_Odds_Spread  48 non-null     float64
dtypes: float64(7)
memory usage: 3.4 KB


In [306]:
full_odds_df = full_odds_df.rename({"Open_Line_ML": "Open_ML"}, axis=1)

In [332]:
def fill_in_nulls(df, column_list, groupings=3):
    book_regex = re.compile(r"Open|Pinnacle|5dimes|Heritage|Bovada|Betonline")
    # Create a Dictionary of Columns of books to their closest correlate book
    correlate_dictionary = {}
    for columns in column_list:
        correlation_matrix = df.dropna().loc[:, columns].corr()
        for column in correlation_matrix.columns:
            correlate_dictionary[column] = correlation_matrix.loc[:, column].sort_values().index[-2]
    
    for index, row in df.iterrows():
        for columns in column_list:
            for column in columns:
                if np.isnan(row[column]) and ("ML" in column): # checks for np.nan and if Moneyline
                    if np.isnan(row[correlate_dictionary[column]]): # checks for associate correlate np.nan
                        df.loc[index, column] = row[column.replace(book_regex.search(column)[0], "Average_Line")] # reverts to average
                    else:
                        df.loc[index, column] = row[correlate_dictionary[column]] # Correlate was not null, row is filled in
                elif np.isnan(row[column]): # checks for np.nan and if number is spread/total 
                    if np.isnan(row[correlate_dictionary[column]]): # checks for associate correlate np.nan
                        df.loc[index, column] = row[column.replace(book_regex.search(column)[0], "Average")] # reverts to average
                        df.loc[index, column.replace("Line", "Odds")] = row[column.replace(book_regex.search(column)[0], "Average").replace("Line", "Odds")]
                    else:
                        df.loc[index, column] = row[correlate_dictionary[column]] # Correlate was not null, number is filled in
                        df.loc[index, column.replace("Line", "Odds")] = row[correlate_dictionary[column].replace("Line", "Odds")] # Correlate was not null, corresponding Odds are filled in
    
    return df

In [328]:
sample_df = full_odds_df.copy(deep=True)

In [329]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5240 entries, 0 to 5239
Data columns (total 59 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   5240 non-null   object 
 1   Location               5240 non-null   object 
 2   Team                   5240 non-null   object 
 3   OppTeam                5240 non-null   object 
 4   TeamId                 5240 non-null   int64  
 5   GameId                 5240 non-null   int64  
 6   PercentBet_ML          5240 non-null   float64
 7   Open_ML                5098 non-null   float64
 8   Pinnacle_ML            5140 non-null   float64
 9   5dimes_ML              5142 non-null   float64
 10  Heritage_ML            5128 non-null   float64
 11  Bovada_ML              5126 non-null   float64
 12  Betonline_ML           5138 non-null   float64
 13  Average_Line_ML        5240 non-null   float64
 14  Best_Line_ML           5240 non-null   float64
 15  Wors

In [333]:
wanted_columns = [['Open_ML', 'Pinnacle_ML', '5dimes_ML','Heritage_ML', 'Bovada_ML', 'Betonline_ML'],
 ['Open_Line_Spread', 'Pinnacle_Line_Spread', '5dimes_Line_Spread', '5dimes_Odds_Spread',
 'Heritage_Line_Spread', 'Bovada_Line_Spread','Betonline_Line_Spread'], 
 ['Open_Line_OU', 'Pinnacle_Line_OU','5dimes_Line_OU', 
 'Heritage_Line_OU', 'Bovada_Line_OU','Betonline_Line_OU']]

In [330]:
new_df = fill_in_nulls(sample_df, wanted_columns)

replaced 550 2
replaced 551 2
replaced 586 2
replaced 587 2
replaced 1286 2
replaced 1287 2
replaced 1922 2
replaced 1922 2
replaced 1923 2
replaced 1923 2
replaced 1982 2
replaced 1982 4
replaced 1983 2
replaced 1983 4
replaced 2262 2
replaced 2263 2
replaced 2304 2
replaced 2304 2
replaced 2305 2
replaced 2305 2
replaced 2306 2
replaced 2306 2
replaced 2307 2
replaced 2307 2
replaced 2394 2
replaced 2394 2
replaced 2395 2
replaced 2395 2
replaced 2402 2
replaced 2403 2
replaced 2404 2
replaced 2404 2
replaced 2405 2
replaced 2405 2
replaced 2452 2
replaced 2453 2
replaced 2624 4
replaced 2625 4
replaced 2628 4
replaced 2629 4
replaced 2630 2
replaced 2631 2
replaced 2634 4
replaced 2635 4
replaced 2635 4
replaced 2636 4
replaced 2637 4
replaced 2638 2
replaced 2639 2
replaced 2642 4
replaced 2643 4
replaced 2644 2
replaced 2644 4
replaced 2645 2
replaced 2645 4
replaced 2650 2
replaced 2651 2
replaced 2692 4
replaced 2693 4
replaced 2694 4
replaced 2695 4
replaced 2710 4
replaced 271

In [331]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5240 entries, 0 to 5239
Data columns (total 59 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   5240 non-null   object 
 1   Location               5240 non-null   object 
 2   Team                   5240 non-null   object 
 3   OppTeam                5240 non-null   object 
 4   TeamId                 5240 non-null   int64  
 5   GameId                 5240 non-null   int64  
 6   PercentBet_ML          5240 non-null   float64
 7   Open_ML                5240 non-null   float64
 8   Pinnacle_ML            5240 non-null   float64
 9   5dimes_ML              5240 non-null   float64
 10  Heritage_ML            5240 non-null   float64
 11  Bovada_ML              5240 non-null   float64
 12  Betonline_ML           5240 non-null   float64
 13  Average_Line_ML        5240 non-null   float64
 14  Best_Line_ML           5240 non-null   float64
 15  Wors

In [ ]:
# Remaining Objectives:
 # Get the Vegas Data on a game by game level as is the current case with g_by_g_df
    # Do this by constructing the high level information in the full odds dataframe to reverse back into the original game_codes we have today
    # I think we can still do this by joining on team and date as well. Which should be easier and quicker.
 # Once we have this we will end up with Sportline information for all games of the the past two full seasons (17-18) and (18-19)
    # I did this first so we could have additional information about the gambling markets to compare to the model we will create (i.e.: A baseline to compare our error to the true market error for each game)

    # So once I get this data to a game by game configuration, I will able to match it with our current game information and start the model building process.  

In [335]:
full_odds_df.Team.unique()

array(['Boston', 'Cleveland', 'Houston', 'Golden State', 'Charlotte',
       'Detroit', 'Brooklyn', 'Indiana', 'Miami', 'Orlando',
       'Philadelphia', 'Washington', 'Milwaukee', 'New Orleans',
       'Memphis', 'Atlanta', 'Dallas', 'Denver', 'Utah', 'Minnesota',
       'San Antonio', 'Portland', 'Phoenix', 'Sacramento', 'Chicago',
       'Toronto', 'New York', 'Oklahoma City', 'L.A. Clippers',
       'L.A. Lakers'], dtype=object)

In [336]:
team_translator = {'Toronto':"TOR", 'Boston' :"BOS", 'Philadelphia':"PHI", 'Brooklyn' :"BRK" , 'New York':"NYK",
            'Denver':"DEN", 'Utah':"UTA", 'Oklahoma City':"OKC", 'Portland' :"POR" , 'Minnesota':"MIN",
             'Milwaukee':"MIL", 'Indiana':"IND", 'Chicago':"CHI", 'Detroit':"DET", 'Cleveland' :"CLE",
             'Miami':"MIA", 'Orlando':"ORL", 'Washington':"WAS", 'Charlotte':"CHO", 'Atlanta' :"ATL",
             'L.A. Lakers':"LAL", 'L.A. Clippers':"LAC", 'Sacramento':"SAC", 'Phoenix':"PHO", 'Golden State' :"GSW",
             'Dallas':"DAL", 'Houston' :"HOU", 'Memphis':"MEM", 'New Orleans':"NOP", 'San Antonio':"SAS"}

In [337]:
df_full_stats.head()

,team,minutes_played,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,usg_pct,off_rtg,def_rtg,pf,pts,home_team,home_team_bin,game_code,date,NBA_season
0,DAL,240.0,38.0,92.0,0.413,12.0,30.0,0.400,19.0,24.0,...,100.0,106.2,115.2,17.0,107.0,TOR,False,201810260TOR,2018-10-26,18-19
1,TOR,240.0,44.0,91.0,0.484,11.0,27.0,0.407,17.0,20.0,...,100.0,115.2,106.2,22.0,116.0,TOR,True,201810260TOR,2018-10-26,18-19
2,BOS,240.0,40.0,99.0,0.404,14.0,36.0,0.389,7.0,10.0,...,100.0,103.3,115.6,27.0,101.0,TOR,False,201810190TOR,2018-10-19,18-19
3,TOR,240.0,43.0,92.0,0.467,13.0,35.0,0.371,14.0,19.0,...,100.0,115.6,103.3,19.0,113.0,TOR,True,201810190TOR,2018-10-19,18-19
4,CLE,240.0,34.0,85.0,0.400,7.0,19.0,0.368,29.0,39.0,...,100.0,103.7,115.6,25.0,104.0,TOR,False,201810170TOR,2018-10-17,18-19


In [339]:
full_odds_df.head()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_ML,Pinnacle_ML,5dimes_ML,...,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,playoffs
0,2017-10-17,away,Boston,Cleveland,1610612738,21700001,50.4,150.0,166.0,170.0,...,-106.83,216.5,216.0,-101.0,-110.0,99.0,-3.0,L,201.0,0
1,2017-10-17,home,Cleveland,Boston,1610612739,21700001,49.6,-170.0,-185.0,-185.0,...,-108.17,216.5,216.0,-105.0,-110.0,102.0,3.0,W,201.0,0
2,2017-10-17,away,Houston,Golden State,1610612745,21700002,52.7,405.0,377.0,420.0,...,-109.67,235.0,230.5,-103.0,-115.0,122.0,1.0,W,243.0,0
3,2017-10-17,home,Golden State,Houston,1610612744,21700002,47.3,-500.0,-440.0,-475.0,...,-105.33,235.0,230.5,-100.0,-110.0,121.0,-1.0,L,243.0,0
4,2017-10-18,away,Charlotte,Detroit,1610612766,21700003,51.3,130.0,127.0,130.0,...,-107.83,203.0,202.0,-102.0,-115.0,90.0,-12.0,L,192.0,0


In [21]:
full_odds_df.sort_values("GameId")

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,playoffs
0,2017-10-17,away,Boston,Cleveland,1610612738,21700001,50.4,150.0,166.0,170.0,...,-106.83,216.5,216.0,-101.0,-110.0,99.0,-3.0,L,201.0,0
1,2017-10-17,home,Cleveland,Boston,1610612739,21700001,49.6,-170.0,-185.0,-185.0,...,-108.17,216.5,216.0,-105.0,-110.0,102.0,3.0,W,201.0,0
2,2017-10-17,away,Houston,Golden State,1610612745,21700002,52.7,405.0,377.0,420.0,...,-109.67,235.0,230.5,-103.0,-115.0,122.0,1.0,W,243.0,0
3,2017-10-17,home,Golden State,Houston,1610612744,21700002,47.3,-500.0,-440.0,-475.0,...,-105.33,235.0,230.5,-100.0,-110.0,121.0,-1.0,L,243.0,0
4,2017-10-18,away,Charlotte,Detroit,1610612766,21700003,51.3,130.0,127.0,130.0,...,-107.83,203.0,202.0,-102.0,-115.0,90.0,-12.0,L,192.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,2019-06-07,home,Golden State,Toronto,1610612744,41800404,50.2,-253.0,-216.0,NaN,...,-105.00,215.5,214.5,-105.0,-105.0,92.0,-13.0,L,197.0,1
5236,2019-06-10,away,Golden State,Toronto,1610612744,41800405,59.7,135.0,-104.0,-105.0,...,-108.17,217.0,211.5,-104.0,-115.0,106.0,1.0,W,211.0,1
5237,2019-06-10,home,Toronto,Golden State,1610612761,41800405,40.3,-155.0,-106.0,-105.0,...,-106.83,217.0,211.5,-105.0,-110.0,105.0,-1.0,L,211.0,1
5238,2019-06-13,away,Toronto,Golden State,1610612761,41800406,54.2,151.0,125.0,125.0,...,-107.50,212.5,211.0,-102.0,-115.0,114.0,4.0,W,224.0,1


In [23]:
def home(location):
    if location == "home":
        return True
    else:
        return False

In [26]:
full_odds_df = full_odds_df.assign(home_team_bin = lambda x: x.Location.apply(home))

In [28]:
full_odds_df = full_odds_df.assign(team_abbv = lambda x: x.Team.apply(lambda x: team_translator[x]))

In [29]:
full_odds_df

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,playoffs,home_team_bin,team_abbv
0,2017-10-17,away,Boston,Cleveland,1610612738,21700001,50.4,150.0,166.0,170.0,...,216.0,-101.0,-110.0,99.0,-3.0,L,201.0,0,False,BOS
1,2017-10-17,home,Cleveland,Boston,1610612739,21700001,49.6,-170.0,-185.0,-185.0,...,216.0,-105.0,-110.0,102.0,3.0,W,201.0,0,True,CLE
2,2017-10-17,away,Houston,Golden State,1610612745,21700002,52.7,405.0,377.0,420.0,...,230.5,-103.0,-115.0,122.0,1.0,W,243.0,0,False,HOU
3,2017-10-17,home,Golden State,Houston,1610612744,21700002,47.3,-500.0,-440.0,-475.0,...,230.5,-100.0,-110.0,121.0,-1.0,L,243.0,0,True,GSW
4,2017-10-18,away,Charlotte,Detroit,1610612766,21700003,51.3,130.0,127.0,130.0,...,202.0,-102.0,-115.0,90.0,-12.0,L,192.0,0,False,CHO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,2019-06-07,home,Golden State,Toronto,1610612744,41800404,50.2,-253.0,-216.0,NaN,...,214.5,-105.0,-105.0,92.0,-13.0,L,197.0,1,True,GSW
5236,2019-06-10,away,Golden State,Toronto,1610612744,41800405,59.7,135.0,-104.0,-105.0,...,211.5,-104.0,-115.0,106.0,1.0,W,211.0,1,False,GSW
5237,2019-06-10,home,Toronto,Golden State,1610612761,41800405,40.3,-155.0,-106.0,-105.0,...,211.5,-105.0,-110.0,105.0,-1.0,L,211.0,1,True,TOR
5238,2019-06-13,away,Toronto,Golden State,1610612761,41800406,54.2,151.0,125.0,125.0,...,211.0,-102.0,-115.0,114.0,4.0,W,224.0,1,False,TOR


In [30]:
for i, row in full_odds_df.iterrows():
    if row.home_team_bin == True:
        full_odds_df.loc[i, "game_code"] = str(row.Date).replace("-", "") + "0" + row.team_abbv

In [34]:
full_odds_df = full_odds_df.sort_values(["GameId", "Location"], ascending=False)

In [35]:
full_odds_df.head()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,playoffs,home_team_bin,team_abbv,game_code
5239,2019-06-13,home,Golden State,Toronto,1610612744,41800406,45.8,-175.0,-138.0,-135.0,...,-105.0,-110.0,110.0,-4.0,L,224.0,1,True,GSW,201906130GSW
5238,2019-06-13,away,Toronto,Golden State,1610612761,41800406,54.2,151.0,125.0,125.0,...,-102.0,-115.0,114.0,4.0,W,224.0,1,False,TOR,NaN
5237,2019-06-10,home,Toronto,Golden State,1610612761,41800405,40.3,-155.0,-106.0,-105.0,...,-105.0,-110.0,105.0,-1.0,L,211.0,1,True,TOR,201906100TOR
5236,2019-06-10,away,Golden State,Toronto,1610612744,41800405,59.7,135.0,-104.0,-105.0,...,-104.0,-115.0,106.0,1.0,W,211.0,1,False,GSW,NaN
5235,2019-06-07,home,Golden State,Toronto,1610612744,41800404,50.2,-253.0,-216.0,NaN,...,-105.0,-105.0,92.0,-13.0,L,197.0,1,True,GSW,201906070GSW


In [38]:
full_odds_df.game_code = full_odds_df.game_code.fillna(method="ffill")

In [44]:
home_odds = full_odds_df[lambda x: x.Location == "home"]
away_odds = full_odds_df[~(full_odds_df.Location == "home")]

In [45]:
pd.merge()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,...,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,playoffs,home_team_bin,team_abbv,game_code
5239,2019-06-13,home,Golden State,Toronto,1610612744,41800406,45.8,-175.0,-138.0,-135.0,...,-105.0,-110.0,110.0,-4.0,L,224.0,1,True,GSW,201906130GSW
5237,2019-06-10,home,Toronto,Golden State,1610612761,41800405,40.3,-155.0,-106.0,-105.0,...,-105.0,-110.0,105.0,-1.0,L,211.0,1,True,TOR,201906100TOR
5235,2019-06-07,home,Golden State,Toronto,1610612744,41800404,50.2,-253.0,-216.0,NaN,...,-105.0,-105.0,92.0,-13.0,L,197.0,1,True,GSW,201906070GSW
5233,2019-06-05,home,Golden State,Toronto,1610612744,41800403,40.3,-220.0,-133.0,-148.0,...,-105.0,-240.0,109.0,-14.0,L,232.0,1,True,GSW,201906050GSW
5231,2019-06-02,home,Toronto,Golden State,1610612761,41800402,34.3,-120.0,NaN,-132.0,...,-103.0,-115.0,104.0,-5.0,L,213.0,1,True,TOR,201906020TOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2017-10-18,home,Orlando,Miami,1610612753,21700005,100.0,130.0,146.0,150.0,...,-103.0,-110.0,116.0,7.0,W,225.0,0,True,ORL,201710180ORL
7,2017-10-18,home,Indiana,Brooklyn,1610612754,21700004,56.9,-155.0,-154.0,-150.0,...,-100.0,-115.0,140.0,9.0,W,271.0,0,True,IND,201710180IND
5,2017-10-18,home,Detroit,Charlotte,1610612765,21700003,48.7,-150.0,-140.0,-140.0,...,-105.0,-110.0,102.0,12.0,W,192.0,0,True,DET,201710180DET
3,2017-10-17,home,Golden State,Houston,1610612744,21700002,47.3,-500.0,-440.0,-475.0,...,-100.0,-110.0,121.0,-1.0,L,243.0,0,True,GSW,201710170GSW


In [47]:
odds_final = pd.merge(home_odds, away_odds, on=["game_code"], suffixes=("_home", "_away"))

In [49]:
odds_final.to_csv(index=False, path_or_buf="../data/interim/vegas_game_by_game_odds.csv")